In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import pandas as pd
from collections import Counter
import nltk
import time
from tqdm import tqdm_notebook
import numpy

In [86]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Working')

INFO:root:Working


In [64]:
_DATA_FILEPATH = '../data/dataninja2019_ads_train.csv'
_POLISH_STOPWORDS_FILEPATH = "../src/polish_stopwords.txt"
_TOKENIZER_FILEPATH = '../src/dataNinjaTokenizer.txt'

In [9]:
df = pd.read_csv(_DATA_FILEPATH)

In [4]:
df.head()

,id,title,description,category_id,region_id,subregion_id,city_id,district_id,params,created_at,labels
0,1432081133,GLIKOL Płyn Chłodniczy Koncentrat Centra. Ogrz...,Nawiąże współpracę z odbiorcami płynów eksploa...,168,13,88,1104129800,NaN,NaN,2012-05-20 23:26:34,glikol_koncentrat wspolpraca koncentrat_do_chl...
1,671872279,Instalacje Gazowe/Przyłącza Gazowe/Sieci gazow...,BARTES świadczy usługi w obejmujące budowę pro...,306,7,136,284937915,1.285174e+09,NaN,2012-06-28 09:00:20,kuchnia_gazowa gazowe przylacza_gazowe przylac...
2,778099925,Elegancki kuferek drewniany + 5 palet na 2 zł ...,Kuferek drewniany z paletami na 200 monet 2 zł...,431,11,260,405646303,6.764402e+07,price<=>price<br>price<=>120,2012-07-09 07:40:37,palety_drewniane
3,1610095002,"Remont łazienki, remont mieszkania, domów, lok...","DROBNE I KOMPLEKSOWE REMONTY MIESZKAŃ , BIUR ,...",306,6,48,689879597,8.987401e+08,NaN,2012-06-27 03:55:33,remont_mieszkania remont_lazienki budowa_domow
4,1884757810,"Karcher Profesjonalne Pranie i czyszczenie, oz...","SAMOCHODY (osobowe, dostawcze, ciężarowe, przy...",354,1,259,1797088492,NaN,NaN,2012-07-04 14:43:56,karcher ozonowanie czyszczenie


In [139]:
class DataNinjaTokenizer:
    stopwords = open(_POLISH_STOPWORDS_FILEPATH).read().split("\n")
    
    def __init__(self, series=None, data_frame=None, headers=None):
        start = time.time()
        assert not (series is None and data_frame is None), "Cannot parse both series and data_frame"
        logger.info('Preparing vocabulary')
        if(series is not None):
            self.__vocabulary = {}
            self.__vocabulary['series'] = self.parse_series_into_words(series)
        if(data_frame is not None):
            assert headers is not None, "Headers are required to tokenize data_frame"
            self.__vocabulary = {}
            for header in headers:
                logger.info('\tProcessing: %s', header)
                if header in self.__vocabulary:
                    self.__vocabulary[header] = self.__vocabulary[header] + self.parse_series_into_words(data_frame[header])
                else:
                    self.__vocabulary[header] = self.parse_series_into_words(data_frame[header])
        assert self.__vocabulary is not None, "Vocabulary cannot be none"
        logger.info('Preparing vocabulary ints')
        self.prepare_vocabulary_ints()
        logger.info('Done in: %0.2f', (time.time() - start))
    
    def parse_series_into_words(self, series):
        words = tqdm_notebook(nltk.word_tokenize(self.series_into_string(series), language="polish"), desc='Word tokenization:')
        words = tqdm_notebook([word for word in words if word.isalpha() and word.lower() not in self.stopwords], desc='Remove alpha and stopwords:')
        return [word.lower() for word in words]
    
    def series_into_string(self, series):
        return ' '.join([row for row in tqdm_notebook(series, desc='Series to string concatenation:')])
                
    def prepare_vocabulary_ints(self):
        self.__vocabulary_ints = {tup:i for i, tup in enumerate(set(self.vocabulary()))}
            
    def vocabulary_frequency(self, headers = None):
        if headers is None:
            return Counter(self.vocabulary())
        else:    
            return {k: Counter(v) for k,v in self.__vocabulary.items() if k in headers }
            
    def vocabulary(self, headers = None):
        if headers is None:
            return [val for vals in self.__vocabulary.values() for val in vals]
        else:
            return {k: v for k, v in self.__vocabulary.items() if k in headers}
    
    @property
    def vocabulary_ints(self):   
        #save to file
        return self.__vocabulary_ints

In [ ]:
temp = DataNinjaTokenizer(series=df['title'][:5])

In [192]:
temp.vocabulary_frequency().most_common(30)

[('x', 4),
 ('kuferek', 3),
 ('koncentrat', 2),
 ('drewniany', 2),
 ('palet', 2),
 ('remont', 2),
 ('lokali', 2),
 ('pranie', 2),
 ('czyszczenie', 2),
 ('gazowych', 2),
 ('wymiar', 2),
 ('mm', 2),
 ('glikol', 1),
 ('płyn', 1),
 ('chłodniczy', 1),
 ('centra', 1),
 ('ogrzew', 1),
 ('solar', 1),
 ('pc', 1),
 ('chłodnica', 1),
 ('instalacje', 1),
 ('elegancki', 1),
 ('ng', 1),
 ('kaplsach', 1),
 ('łazienki', 1),
 ('mieszkania', 1),
 ('domów', 1),
 ('klatek', 1),
 ('schodowych', 1),
 ('karcher', 1)]

In [184]:
temp.vocabulary()

['glikol',
 'płyn',
 'chłodniczy',
 'koncentrat',
 'centra',
 'ogrzew',
 'solar',
 'pc',
 'chłodnica',
 'instalacje',
 'elegancki',
 'kuferek',
 'drewniany',
 'palet',
 'ng',
 'kaplsach',
 'remont',
 'łazienki',
 'remont',
 'mieszkania',
 'domów',
 'lokali',
 'klatek',
 'schodowych',
 'karcher',
 'profesjonalne',
 'pranie',
 'czyszczenie',
 'ozonowanie']

In [140]:
temp = DataNinjaTokenizer(data_frame=df[:200000], headers=['title', 'description'])

INFO:root:Preparing vocabulary
INFO:root:	Processing: title


HBox(children=(IntProgress(value=0, description='Series to string concatenation:', max=200000, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Word tokenization:', max=1310022, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Remove alpha and stopwords:', max=888628, style=ProgressStyle(description_width='initial')), HTML(value='')))

INFO:root:	Processing: description


HBox(children=(IntProgress(value=0, description='Series to string concatenation:', max=200000, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Word tokenization:', max=6375575, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Remove alpha and stopwords:', max=3548040, style=ProgressStyle(description_width='initial')), HTML(value='')))

INFO:root:Preparing vocabulary ints
INFO:root:Done in: 151.88


In [127]:
temp.vocabulary_frequency(headers=['title', 'description'])['title'].most_common(20)

[('city', 9383),
 ('rozmiar', 3933),
 ('rozm', 3059),
 ('ślubna', 2792),
 ('nowe', 2667),
 ('nowa', 2428),
 ('surname', 2325),
 ('m', 2261),
 ('name', 2208),
 ('sukienka', 2058),
 ('nowy', 1985),
 ('kurtka', 1886),
 ('roz', 1773),
 ('buty', 1488),
 ('stan', 1476),
 ('suknia', 1453),
 ('firmy', 1428),
 ('cm', 1388),
 ('skóra', 1353),
 ('okazja', 1342)]

In [190]:
import pickle

In [ ]:
filehandler = open(_TOKENIZER_FILEPATH, 'wb')
pickle.dump(temp, filehandler)

In [218]:
filehandler = open(_TOKENIZER_FILEPATH, 'rb')
temp1 = pickle.load(filehandler)